# [문제]
- law_2.docx 파일을 읽고, Chroma 저장
- LLM에 질문 -> 답변
- '전제사기피해'에 관한 법률 질문만 받기
- 이 외의 질문을 하면 '답변을 할 수 없습니다.'

In [1]:
# 필요한 모듈
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain

In [2]:
# 문서 읽고 Chroma에 저장
loader = Docx2txtLoader('law_2.docx')
documents = loader.load()

In [3]:
# 문서 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
)
docs = text_splitter.split_documents(documents)

In [4]:
# 벡터 DB에 저장
vector_db = Chroma.from_documents(
    docs,
    OpenAIEmbeddings(),
    persist_directory='./chroma_db'
)

In [5]:
# RAG QA Chain 생성
retriever = vector_db.as_retriever()
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model='gpt-4o'),
    retriever=retriever,
    return_source_documents=True
)

In [6]:
# 질문 받기
while True:
    query = input("전세사기피해에 대해 질문하세요 (종료하려면 'q' 입력): ")
    
    if query.lower() == 'q':
        print("종료합니다.")
        break

    if '전세사기피해' in query:
        # LLM에 질문과 함께 빈 chat_history 전달
        result = qa_chain.invoke({
            'question': query,
            'chat_history': []
        })
        
        # 결과 출력
        print('\n📌 답변:', result.get('answer', '답변 없음'))

        # 소스 문서도 확인하고 싶다면 아래처럼 출력 가능
        for idx, doc in enumerate(result.get('source_documents', [])):
            print(f"\n🔗 소스 {idx+1}:\n", doc.page_content)
    else:
        print("❗️답변을 할 수 없습니다.")



📌 답변: 전세사기피해는 주택임대차 계약에서 발생하는 사기로, 주로 임차인이 전세금을 잃게 되는 상황을 의미합니다. 이는 집주인이 임차인에게 전세금을 받아 놓고도 주택담보대출을 갚지 않거나 주택을 임의로 매각하여 임차인에게 상환되지 않는 경우 등을 포함할 수 있습니다. 이러한 사기는 임차인의 경제적 손실을 초래하며, 전세사기피해자를 보호히기 위한 특별법 및 시행령이 마련되어 있기도 합니다.

🔗 소스 1:
 전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )

🔗 소스 2:
 전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 ( 약칭: 전세사기피해자법 시행령 )

🔗 소스 3:
 1. 전세사기피해주택 매입금액

🔗 소스 4:
 1. 전세사기피해주택 매입금액
❗️답변을 할 수 없습니다.
❗️답변을 할 수 없습니다.
❗️답변을 할 수 없습니다.
종료합니다.
